In [1]:
import pandas as pd

In [2]:
# url > https://dados-ml-pln.s3.sa-east-1.amazonaws.com/tickets_reclamacoes_classificados.csv
df = pd.read_csv('https://dados-ml-pln.s3.sa-east-1.amazonaws.com/tickets_reclamacoes_classificados.csv', delimiter=';')
df.head()

,id_reclamacao,data_abertura,categoria,descricao_reclamacao
0,3229299,2019-05-01T12:00:00-05:00,Hipotecas / Empréstimos,"Bom dia, meu nome é xxxx xxxx e agradeço se vo..."
1,3199379,2019-04-02T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,Atualizei meu cartão xxxx xxxx em xx/xx/2018 e...
2,3233499,2019-05-06T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,O cartão Chase foi relatado em xx/xx/2019. No ...
3,3180294,2019-03-14T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,"Em xx/xx/2018, enquanto tentava reservar um ti..."
4,3224980,2019-04-27T12:00:00-05:00,Serviços de conta bancária,"Meu neto me dê cheque por {$ 1600,00} Eu depos..."


## Remoção de nulos se houver

In [3]:
df.isnull().sum()

id_reclamacao           0
data_abertura           0
categoria               0
descricao_reclamacao    0
dtype: int64

## Distribuição do target

In [4]:
df['categoria'].value_counts() / df.shape[0]

Serviços de conta bancária             0.244922
Cartão de crédito / Cartão pré-pago    0.237566
Roubo / Relatório de disputa           0.228834
Hipotecas / Empréstimos                0.182707
Outros                                 0.105970
Name: categoria, dtype: float64

## Modelos baseline
### Préprocessamento

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vinic\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
# Neste primeiro teste opitamos por apenas fazer uma vetorização unigram removendo stop words
stop_words = nltk.corpus.stopwords.words('portuguese')
print(stop_words)

['a', 'à', 'ao', 'aos', 'aquela', 'aquelas', 'aquele', 'aqueles', 'aquilo', 'as', 'às', 'até', 'com', 'como', 'da', 'das', 'de', 'dela', 'delas', 'dele', 'deles', 'depois', 'do', 'dos', 'e', 'é', 'ela', 'elas', 'ele', 'eles', 'em', 'entre', 'era', 'eram', 'éramos', 'essa', 'essas', 'esse', 'esses', 'esta', 'está', 'estamos', 'estão', 'estar', 'estas', 'estava', 'estavam', 'estávamos', 'este', 'esteja', 'estejam', 'estejamos', 'estes', 'esteve', 'estive', 'estivemos', 'estiver', 'estivera', 'estiveram', 'estivéramos', 'estiverem', 'estivermos', 'estivesse', 'estivessem', 'estivéssemos', 'estou', 'eu', 'foi', 'fomos', 'for', 'fora', 'foram', 'fôramos', 'forem', 'formos', 'fosse', 'fossem', 'fôssemos', 'fui', 'há', 'haja', 'hajam', 'hajamos', 'hão', 'havemos', 'haver', 'hei', 'houve', 'houvemos', 'houver', 'houvera', 'houverá', 'houveram', 'houvéramos', 'houverão', 'houverei', 'houverem', 'houveremos', 'houveria', 'houveriam', 'houveríamos', 'houvermos', 'houvesse', 'houvessem', 'houvésse

In [7]:
df['categoria'].unique()

array(['Hipotecas / Empréstimos', 'Cartão de crédito / Cartão pré-pago',
       'Serviços de conta bancária', 'Outros',
       'Roubo / Relatório de disputa'], dtype=object)

In [8]:
vetorizador = CountVectorizer(ngram_range=(1,1))
vetorizador.fit(df['descricao_reclamacao'])
vect = vetorizador.transform(df['descricao_reclamacao'])
vect

<21072x42401 sparse matrix of type '<class 'numpy.int64'>'
	with 2491849 stored elements in Compressed Sparse Row format>

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

y = df['categoria']
x_train, x_test, y_train, y_test = train_test_split(vect, y, test_size=0.25, random_state=42)

In [20]:
from sklearn.metrics import f1_score
def gera_resultado(y_train, y_train_true, y_test, y_test_true):
    avaliacao_train = f1_score(y_train_true, y_train, average='weighted')
    avaliacao_test = f1_score(y_test_true, y_test, average='weighted')
    
    print(f"Treino: {avaliacao_train}" )
    print(f"Teste: {avaliacao_test}")

### Decision Tree

In [11]:
from sklearn.tree import DecisionTreeClassifier
modelo = DecisionTreeClassifier()
modelo.fit(x_train, y_train)
preds = modelo.predict(x_train)
preds_test = modelo.predict(x_test)

In [12]:
gera_resultado(preds, y_train, preds_test, y_test)

Treino: 1.0
Teste: 0.7254530125641099


### Random Forest

In [13]:
modelo = RandomForestClassifier()
modelo.fit(x_train, y_train)
preds = modelo.predict(x_train)
preds_test = modelo.predict(x_test)

gera_resultado(preds, y_train, preds_test, y_test)

Treino: 1.0
Teste: 0.7936708642534053


### Naive Bayes

In [17]:
from sklearn.naive_bayes import GaussianNB
import numpy as np

modelo = GaussianNB()
modelo.fit(np.asarray(x_train.todense()), y_train)
preds = modelo.predict(np.asarray(x_train.todense()))
preds_test = modelo.predict(np.asarray(x_test.todense()))

gera_resultado(preds, y_train, preds_test, y_test)

Treino: 0.7919900649682384
Teste: 0.3989893166443923


### MLP

In [13]:
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras import Input
from tensorflow.keras.models import Sequential
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [7]:
model = Sequential()

model.add(Dense(64, activation='relu', input_dim=42401))
model.add(Dense(32, activation='relu'))
model.add(Dense(5, activation='softmax'))

model.compile(loss="categorical_crossentropy", optimizer='adam', metrics="accuracy")

In [15]:
es = EarlyStopping(monitor='val_loss', verbose=1, patience=18)
mc = ModelCheckpoint('best_model_initial.h5', monitor='val_loss', mode='min', save_best_only=True)
model.fit(np.asarray(x_train.todense()), pd.get_dummies(y_train.values).values,
          validation_data=(np.asarray(x_test.todense()), pd.get_dummies(y_test.values).values), 
          epochs=30,
          callbacks=[es, mc])

Epoch 1/30
494/494 [==============================] - 9s 17ms/step - loss: 0.5516 - accuracy: 0.8283 - val_loss: 0.4731 - val_accuracy: 0.8535
Epoch 2/30
494/494 [==============================] - 6s 12ms/step - loss: 0.1640 - accuracy: 0.9537 - val_loss: 0.3974 - val_accuracy: 0.8749
Epoch 3/30
494/494 [==============================] - 6s 12ms/step - loss: 0.0598 - accuracy: 0.9848 - val_loss: 0.4677 - val_accuracy: 0.8762
Epoch 4/30
494/494 [==============================] - 6s 12ms/step - loss: 0.0668 - accuracy: 0.9839 - val_loss: 0.5346 - val_accuracy: 0.8700
Epoch 5/30
494/494 [==============================] - 6s 13ms/step - loss: 0.0281 - accuracy: 0.9941 - val_loss: 0.5365 - val_accuracy: 0.8795
Epoch 6/30
494/494 [==============================] - 6s 12ms/step - loss: 0.0258 - accuracy: 0.9948 - val_loss: 0.6375 - val_accuracy: 0.8707
Epoch 7/30
494/494 [==============================] - 6s 12ms/step - loss: 0.0176 - accuracy: 0.9956 - val_loss: 0.9016 - val_accuracy: 0.8481

In [16]:
from tensorflow.keras.models import load_model
model2 = load_model('best_model_initial.h5')

preds = model2.predict(np.asarray(x_train.todense())).argmax(axis=-1)
preds_test = model2.predict(np.asarray(x_test.todense())).argmax(axis=-1)

rotulos = {"Cartão de crédito / Cartão pré-pago": 0, "Hipotecas / Empréstimos": 1, "Outros": 2, 
           "Roubo / Relatório de disputa": 3, "Serviços de conta bancária": 4}

gera_resultado(preds, y_train.map(rotulos), preds_test, y_test.map(rotulos))

165/165 [==============================] - 1s 4ms/step
Treino: 0.9863912482699
Teste: 0.8746002829544764


## Modelo 2 - Adicionando lemmatização aos dados

In [33]:
stop_words.append("/r")
stop_words.append("/n")
stop_words.append("/r/n")
stop_words.append("/n/r")

In [11]:
from nltk.stem.rslp import RSLPStemmer
nltk.download('rslp')
from nltk.tokenize import word_tokenize

df['tokens'] = df['descricao_reclamacao'].apply(word_tokenize)
rslp = RSLPStemmer()

[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\vinic\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


In [10]:
def stem_pandas(line):
  return ' '.join([rslp.stem(token) for token in line])

df['stemmer'] = df.tokens.apply(stem_pandas)

In [74]:
vetorizador = CountVectorizer(ngram_range=(1,1))
vetorizador.fit(df['stemmer'])
vect = vetorizador.transform(df['stemmer'])
vect

<21072x20060 sparse matrix of type '<class 'numpy.int64'>'
	with 2198261 stored elements in Compressed Sparse Row format>

In [79]:
df

,id_reclamacao,data_abertura,categoria,descricao_reclamacao,tokens,stemmer
0,3229299,2019-05-01T12:00:00-05:00,Hipotecas / Empréstimos,"Bom dia, meu nome é xxxx xxxx e agradeço se vo...","[Bom, dia, ,, meu, nome, é, xxxx, xxxx, e, agr...","bom dia , meu nom é xxxx xxxx e agradeç se voc..."
1,3199379,2019-04-02T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,Atualizei meu cartão xxxx xxxx em xx/xx/2018 e...,"[Atualizei, meu, cartão, xxxx, xxxx, em, xx/xx...",atualiz meu cart xxxx xxxx em xx/xx/2018 e fui...
2,3233499,2019-05-06T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,O cartão Chase foi relatado em xx/xx/2019. No ...,"[O, cartão, Chase, foi, relatado, em, xx/xx/20...",o cart chas foi relat em xx/xx/2019 . no entan...
3,3180294,2019-03-14T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,"Em xx/xx/2018, enquanto tentava reservar um ti...","[Em, xx/xx/2018, ,, enquanto, tentava, reserva...","em xx/xx/2018 , enquant tent reserv um ticket ..."
4,3224980,2019-04-27T12:00:00-05:00,Serviços de conta bancária,"Meu neto me dê cheque por {$ 1600,00} Eu depos...","[Meu, neto, me, dê, cheque, por, {, $, 1600,00...","meu net me dê chequ por { $ 1600,00 } eu depos..."
...,...,...,...,...,...,...
21067,3094545,2018-12-07T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,Depois de ser um cliente de cartão de persegui...,"[Depois, de, ser, um, cliente, de, cartão, de,...",depois de ser um client de cart de persegu por...
21068,3091984,2018-12-05T12:00:00-05:00,Roubo / Relatório de disputa,"Na quarta -feira, xx/xx/xxxx, liguei para o Ch...","[Na, quarta, -feira, ,, xx/xx/xxxx, ,, liguei,...","na quart -fe , xx/xx/xxxx , lig par o cha , me..."
21069,3133355,2019-01-25T12:00:00-05:00,Roubo / Relatório de disputa,Não estou familiarizado com o XXXX Pay e não e...,"[Não, estou, familiarizado, com, o, XXXX, Pay,...",não est familiar com o xxxx pay e não entend o...
21070,3110963,2018-12-27T12:00:00-05:00,Outros,Eu tive crédito impecável por 30 anos. Eu tive...,"[Eu, tive, crédito, impecável, por, 30, anos, ...",eu tiv crédit impec por 30 ano . eu tiv cart d...


In [81]:
x_train

<15804x20060 sparse matrix of type '<class 'numpy.int64'>'
	with 1640209 stored elements in Compressed Sparse Row format>

In [83]:
## Vou utilizar o melhor modelo do primeiro teste
y = df['categoria']
x_train, x_test, y_train, y_test = train_test_split(vect, y, test_size=0.25, random_state=42)

model = Sequential()

model.add(Dense(64, activation='relu', input_dim=20060))
model.add(Dense(32, activation='relu'))
model.add(Dense(5, activation='softmax'))

model.compile(loss="categorical_crossentropy", optimizer='adam', metrics="accuracy")

es = EarlyStopping(monitor='val_loss', verbose=1, patience=18)
mc = ModelCheckpoint('best_model_2.h5', monitor='val_loss', mode='min', save_best_only=True)
model.fit(np.asarray(x_train.todense()), pd.get_dummies(y_train.values).values,
          validation_data=(np.asarray(x_test.todense()), pd.get_dummies(y_test.values).values), 
          epochs=30,
          callbacks=[es, mc])

Epoch 1/30
494/494 [==============================] - 4s 7ms/step - loss: 0.5785 - accuracy: 0.8214 - val_loss: 0.4046 - val_accuracy: 0.8597
Epoch 2/30
494/494 [==============================] - 3s 6ms/step - loss: 0.2536 - accuracy: 0.9146 - val_loss: 0.3898 - val_accuracy: 0.8686
Epoch 3/30
494/494 [==============================] - 3s 6ms/step - loss: 0.1544 - accuracy: 0.9480 - val_loss: 0.4834 - val_accuracy: 0.8550
Epoch 4/30
494/494 [==============================] - 3s 6ms/step - loss: 0.1082 - accuracy: 0.9659 - val_loss: 0.4624 - val_accuracy: 0.8688
Epoch 5/30
494/494 [==============================] - 3s 6ms/step - loss: 0.0832 - accuracy: 0.9735 - val_loss: 0.5835 - val_accuracy: 0.8599
Epoch 6/30
494/494 [==============================] - 3s 6ms/step - loss: 0.0585 - accuracy: 0.9820 - val_loss: 0.6330 - val_accuracy: 0.8599
Epoch 7/30
494/494 [==============================] - 3s 6ms/step - loss: 0.0970 - accuracy: 0.9743 - val_loss: 0.5965 - val_accuracy: 0.8514
Epoch 

In [84]:
from tensorflow.keras.models import load_model
model2 = load_model('best_model_2.h5')

preds = model2.predict(np.asarray(x_train.todense())).argmax(axis=-1)
preds_test = model2.predict(np.asarray(x_test.todense())).argmax(axis=-1)

rotulos = {"Cartão de crédito / Cartão pré-pago": 0, "Hipotecas / Empréstimos": 1, "Outros": 2, 
           "Roubo / Relatório de disputa": 3, "Serviços de conta bancária": 4}

gera_resultado(preds, y_train.map(rotulos), preds_test, y_test.map(rotulos))

165/165 [==============================] - 1s 3ms/step
Treino: 0.9626598875590856
Teste: 0.8686972577779358


## Unindo bigram à stemmização
---

In [10]:
vetorizador = CountVectorizer(ngram_range=(2,2))
vetorizador.fit(df['stemmer'])
vect = vetorizador.transform(df['stemmer'])
vect

<21072x504625 sparse matrix of type '<class 'numpy.int64'>'
	with 4053009 stored elements in Compressed Sparse Row format>

In [15]:
vect.shape

(21072, 504625)

In [16]:
## Vou utilizar o melhor modelo do primeiro teste
from sklearn.model_selection import train_test_split
y = df['categoria']
x_train, x_test, y_train, y_test = train_test_split(vect, y, test_size=0.25, random_state=42)


model = Sequential()

model.add(Dense(64, activation='relu', input_dim=504625))
model.add(Dense(32, activation='relu'))
model.add(Dense(5, activation='softmax'))

model.compile(loss="categorical_crossentropy", optimizer='adam', metrics="accuracy")

es = EarlyStopping(monitor='val_loss', verbose=1, patience=18)
mc = ModelCheckpoint('best_model_3.h5', monitor='val_loss', mode='min', save_best_only=True)
model.fit(np.asarray(x_train.todense()), pd.get_dummies(y_train.values).values,
          validation_data=(np.asarray(x_test.todense()), pd.get_dummies(y_test.values).values), 
          epochs=30,
          callbacks=[es, mc])

## Não consegui rodar... 

MemoryError: Unable to allocate 59.4 GiB for an array with shape (15804, 504625) and data type int64

## Testando com algumas classes gramáticais apenas

- Adj
- Verb
- subs

In [9]:
import spacy
pt = spacy.load('pt_core_news_lg')

def helper_pos_tag(texto):
    pos_tag = [str(token) if str(token.pos_) in ["ADJ", 'NOUN', 'VERB'] else '' for token in pt(texto)]
    pos_tag = list(filter(lambda valor: valor if valor != '' else None, pos_tag))
    
    return " ".join(pos_tag)

df['pos_tag'] = df['descricao_reclamacao'].apply(helper_pos_tag)

In [10]:
df['pos_tag'].iloc[0]

'Bom dia nome agradeço puder ajudar acabar serviços membro cartão bancário 2018 escrevi solicitar verificação dívida enviaram declaração aceitável pedindo banco valide dívida vez recebi meses tentando coletar dívida Tenho direito conhecer informações consumidor Conta Obrigado ajuda'

In [8]:
df.to_csv('df_pos_tag.csv', index=False)

In [11]:
vetorizador = CountVectorizer(ngram_range=(1,1))
vetorizador.fit(df['pos_tag'])
vect = vetorizador.transform(df['pos_tag'])
vect

<21072x32345 sparse matrix of type '<class 'numpy.int64'>'
	with 1459384 stored elements in Compressed Sparse Row format>

In [14]:
from sklearn.model_selection import train_test_split
y = df['categoria']
x_train, x_test, y_train, y_test = train_test_split(vect, y, test_size=0.25, random_state=42)


model = Sequential()

model.add(Dense(64, activation='relu', input_dim=32345))
model.add(Dropout(.3))
model.add(Dense(32, activation='relu'))
model.add(Dropout(.3))
model.add(Dense(5, activation='softmax'))

model.compile(loss="categorical_crossentropy", optimizer='adam', metrics="accuracy")

es = EarlyStopping(monitor='val_loss', verbose=1, patience=18)
mc = ModelCheckpoint('best_model_3.h5', monitor='val_loss', mode='min', save_best_only=True)
model.fit(np.asarray(x_train.todense()), pd.get_dummies(y_train.values).values,
          validation_data=(np.asarray(x_test.todense()), pd.get_dummies(y_test.values).values), 
          epochs=30,
          callbacks=[es, mc])

Epoch 1/30
494/494 [==============================] - 4s 7ms/step - loss: 0.6407 - accuracy: 0.7978 - val_loss: 0.3482 - val_accuracy: 0.8869
Epoch 2/30
494/494 [==============================] - 3s 7ms/step - loss: 0.2596 - accuracy: 0.9238 - val_loss: 0.3461 - val_accuracy: 0.8880
Epoch 3/30
494/494 [==============================] - 3s 7ms/step - loss: 0.1449 - accuracy: 0.9586 - val_loss: 0.4087 - val_accuracy: 0.8787
Epoch 4/30
494/494 [==============================] - 3s 7ms/step - loss: 0.0877 - accuracy: 0.9748 - val_loss: 0.4513 - val_accuracy: 0.8812
Epoch 5/30
494/494 [==============================] - 3s 7ms/step - loss: 0.0688 - accuracy: 0.9813 - val_loss: 0.5012 - val_accuracy: 0.8787
Epoch 6/30
494/494 [==============================] - 3s 7ms/step - loss: 0.0531 - accuracy: 0.9861 - val_loss: 0.5596 - val_accuracy: 0.8772
Epoch 7/30
494/494 [==============================] - 3s 7ms/step - loss: 0.0393 - accuracy: 0.9896 - val_loss: 0.6804 - val_accuracy: 0.8717
Epoch 

## Usando frequencia

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer(ngram_range=(1,1), use_idf=False, norm='l1', stop_words=stop_words)
vect.fit(df['pos_tag'])
vect = vect.transform(df['pos_tag'])
vect

<21072x32655 sparse matrix of type '<class 'numpy.float64'>'
	with 1434794 stored elements in Compressed Sparse Row format>

In [57]:
y = df['categoria']
x_train, x_test, y_train, y_test = train_test_split(vect, y, test_size=0.25, random_state=42)


model = Sequential()

model.add(Dense(64, activation='relu', input_dim=32655))
model.add(Dropout(.3))
model.add(Dense(32, activation='relu'))
model.add(Dropout(.3))
model.add(Dense(5, activation='softmax'))

model.compile(loss="categorical_crossentropy", optimizer='adam', metrics="accuracy")

es = EarlyStopping(monitor='val_loss', verbose=1, patience=18)
mc = ModelCheckpoint('best_model_4.h5', monitor='val_loss', mode='min', save_best_only=True)
model.fit(np.asarray(x_train.todense()), pd.get_dummies(y_train.values).values,
          validation_data=(np.asarray(x_test.todense()), pd.get_dummies(y_test.values).values), 
          epochs=30,
          callbacks=[es, mc])

Epoch 1/30
494/494 [==============================] - 5s 9ms/step - loss: 1.1229 - accuracy: 0.5967 - val_loss: 0.5655 - val_accuracy: 0.8229
Epoch 2/30
494/494 [==============================] - 4s 8ms/step - loss: 0.4712 - accuracy: 0.8478 - val_loss: 0.3370 - val_accuracy: 0.8890
Epoch 3/30
494/494 [==============================] - 4s 7ms/step - loss: 0.3048 - accuracy: 0.9046 - val_loss: 0.3027 - val_accuracy: 0.8962
Epoch 4/30
494/494 [==============================] - 4s 7ms/step - loss: 0.2226 - accuracy: 0.9334 - val_loss: 0.2953 - val_accuracy: 0.9003
Epoch 5/30
494/494 [==============================] - 4s 7ms/step - loss: 0.1662 - accuracy: 0.9536 - val_loss: 0.3019 - val_accuracy: 0.8965
Epoch 6/30
494/494 [==============================] - 4s 7ms/step - loss: 0.1249 - accuracy: 0.9650 - val_loss: 0.3164 - val_accuracy: 0.8948
Epoch 7/30
494/494 [==============================] - 4s 7ms/step - loss: 0.0933 - accuracy: 0.9757 - val_loss: 0.3360 - val_accuracy: 0.8933
Epoch 

In [58]:
from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer(ngram_range=(1,1), use_idf=True, norm='l1', stop_words=stop_words)
vect.fit(df['pos_tag'])
vect = vect.transform(df['pos_tag'])
vect

<21072x32655 sparse matrix of type '<class 'numpy.float64'>'
	with 1434794 stored elements in Compressed Sparse Row format>

In [59]:
y = df['categoria']
x_train, x_test, y_train, y_test = train_test_split(vect, y, test_size=0.25, random_state=42)


model = Sequential()

model.add(Dense(64, activation='relu', input_dim=32655))
model.add(Dropout(.3))
model.add(Dense(32, activation='relu'))
model.add(Dropout(.3))
model.add(Dense(5, activation='softmax'))

model.compile(loss="categorical_crossentropy", optimizer='adam', metrics="accuracy")

es = EarlyStopping(monitor='val_loss', verbose=1, patience=18)
mc = ModelCheckpoint('best_model_5.h5', monitor='val_loss', mode='min', save_best_only=True)
model.fit(np.asarray(x_train.todense()), pd.get_dummies(y_train.values).values,
          validation_data=(np.asarray(x_test.todense()), pd.get_dummies(y_test.values).values), 
          epochs=30,
          callbacks=[es, mc])

Epoch 1/30
494/494 [==============================] - 5s 10ms/step - loss: 1.2685 - accuracy: 0.5240 - val_loss: 0.7063 - val_accuracy: 0.8013
Epoch 2/30
494/494 [==============================] - 4s 8ms/step - loss: 0.5394 - accuracy: 0.8328 - val_loss: 0.3998 - val_accuracy: 0.8703
Epoch 3/30
494/494 [==============================] - 4s 7ms/step - loss: 0.3104 - accuracy: 0.9066 - val_loss: 0.3497 - val_accuracy: 0.8772
Epoch 4/30
494/494 [==============================] - 4s 7ms/step - loss: 0.1988 - accuracy: 0.9454 - val_loss: 0.3469 - val_accuracy: 0.8802
Epoch 5/30
494/494 [==============================] - 4s 7ms/step - loss: 0.1334 - accuracy: 0.9651 - val_loss: 0.3573 - val_accuracy: 0.8781
Epoch 6/30
494/494 [==============================] - 4s 7ms/step - loss: 0.0940 - accuracy: 0.9772 - val_loss: 0.3758 - val_accuracy: 0.8743
Epoch 7/30
494/494 [==============================] - 4s 7ms/step - loss: 0.0657 - accuracy: 0.9859 - val_loss: 0.3970 - val_accuracy: 0.8740
Epoch

## Testando TF na stemmização
---

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer(ngram_range=(1,1), use_idf=True, norm='l1', stop_words=stop_words)
vect.fit(df['stemmer'])
vect = vect.transform(df['stemmer'])
vect

KeyError: 'stemmer'

In [61]:
y = df['categoria']
x_train, x_test, y_train, y_test = train_test_split(vect, y, test_size=0.25, random_state=42)


model = Sequential()

model.add(Dense(64, activation='relu', input_dim=19983))
model.add(Dropout(.3))
model.add(Dense(32, activation='relu'))
model.add(Dropout(.3))
model.add(Dense(5, activation='softmax'))

model.compile(loss="categorical_crossentropy", optimizer='adam', metrics="accuracy")

es = EarlyStopping(monitor='val_loss', verbose=1, patience=18)
mc = ModelCheckpoint('best_model_5.h5', monitor='val_loss', mode='min', save_best_only=True)
model.fit(np.asarray(x_train.todense()), pd.get_dummies(y_train.values).values,
          validation_data=(np.asarray(x_test.todense()), pd.get_dummies(y_test.values).values), 
          epochs=30,
          callbacks=[es, mc])

Epoch 1/30
494/494 [==============================] - 3s 5ms/step - loss: 1.2393 - accuracy: 0.5237 - val_loss: 0.7815 - val_accuracy: 0.7563
Epoch 2/30
494/494 [==============================] - 2s 4ms/step - loss: 0.6569 - accuracy: 0.7801 - val_loss: 0.5093 - val_accuracy: 0.8311
Epoch 3/30
494/494 [==============================] - 2s 4ms/step - loss: 0.4332 - accuracy: 0.8600 - val_loss: 0.4138 - val_accuracy: 0.8563
Epoch 4/30
494/494 [==============================] - 2s 4ms/step - loss: 0.3161 - accuracy: 0.8997 - val_loss: 0.3884 - val_accuracy: 0.8641
Epoch 5/30
494/494 [==============================] - 2s 5ms/step - loss: 0.2390 - accuracy: 0.9263 - val_loss: 0.3809 - val_accuracy: 0.8637
Epoch 6/30
494/494 [==============================] - 2s 5ms/step - loss: 0.1949 - accuracy: 0.9420 - val_loss: 0.3840 - val_accuracy: 0.8660
Epoch 7/30
494/494 [==============================] - 2s 4ms/step - loss: 0.1584 - accuracy: 0.9529 - val_loss: 0.4005 - val_accuracy: 0.8635
Epoch 

## Testando com os tokens

In [64]:
from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer(ngram_range=(1,1), use_idf=True, norm='l1', stop_words=stop_words)
vect.fit(df['descricao_reclamacao'])
vect = vect.transform(df['descricao_reclamacao'])
vect

<21072x42226 sparse matrix of type '<class 'numpy.float64'>'
	with 1847318 stored elements in Compressed Sparse Row format>

In [66]:
y = df['categoria']
x_train, x_test, y_train, y_test = train_test_split(vect, y, test_size=0.25, random_state=42)


model = Sequential()

model.add(Dense(64, activation='relu', input_dim=42226))
model.add(Dropout(.3))
model.add(Dense(32, activation='relu'))
model.add(Dropout(.3))
model.add(Dense(5, activation='softmax'))

model.compile(loss="categorical_crossentropy", optimizer='adam', metrics="accuracy")

es = EarlyStopping(monitor='val_loss', verbose=1, patience=18)
mc = ModelCheckpoint('best_model_6.h5', monitor='val_loss', mode='min', save_best_only=True)
model.fit(np.asarray(x_train.todense()), pd.get_dummies(y_train.values).values,
          validation_data=(np.asarray(x_test.todense()), pd.get_dummies(y_test.values).values), 
          epochs=30,
          callbacks=[es, mc])

Epoch 1/30
494/494 [==============================] - 6s 11ms/step - loss: 1.2748 - accuracy: 0.5159 - val_loss: 0.7466 - val_accuracy: 0.7703
Epoch 2/30
494/494 [==============================] - 5s 10ms/step - loss: 0.5879 - accuracy: 0.8080 - val_loss: 0.4595 - val_accuracy: 0.8432
Epoch 3/30
494/494 [==============================] - 5s 10ms/step - loss: 0.3417 - accuracy: 0.8989 - val_loss: 0.3699 - val_accuracy: 0.8645
Epoch 4/30
494/494 [==============================] - 5s 9ms/step - loss: 0.2178 - accuracy: 0.9401 - val_loss: 0.3565 - val_accuracy: 0.8666
Epoch 5/30
494/494 [==============================] - 5s 10ms/step - loss: 0.1429 - accuracy: 0.9628 - val_loss: 0.3564 - val_accuracy: 0.8738
Epoch 6/30
494/494 [==============================] - 5s 9ms/step - loss: 0.1003 - accuracy: 0.9767 - val_loss: 0.3813 - val_accuracy: 0.8679
Epoch 7/30
494/494 [==============================] - 5s 9ms/step - loss: 0.0736 - accuracy: 0.9834 - val_loss: 0.3916 - val_accuracy: 0.8683
Ep

## Montando rede neural e definir modelo campeão

In [15]:
import pandas as pd
from tensorflow.keras.layers import LSTM, Embedding
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras import Input
from tensorflow.keras.models import Sequential
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
# df = pd.read_csv('df_pos_tag.csv')

In [3]:
## Removendo sequência de consoantes
import re

def remove_sequencia_de_consoantes(texto):
    regex = re.sub(r"(?!aeiouâãáíóõôú)[^aeiouâãáíóõôú.?!, ]{4,}(?!aeiouâãáíóõôú)", r" ", texto)
    return regex

df['pos_tag_v2'] = df['pos_tag'].apply(lambda texto: remove_sequencia_de_consoantes(texto.lower()))

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
df.dropna(inplace=True)
vect = TfidfVectorizer(ngram_range=(1,1), use_idf=False, norm='l1')
vect.fit(df['pos_tag_v2'])
vect = vect.transform(df['pos_tag_v2'])
vect

<21071x31821 sparse matrix of type '<class 'numpy.float64'>'
	with 1454234 stored elements in Compressed Sparse Row format>

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
df.dropna(inplace=True)
vect = TfidfVectorizer(ngram_range=(1,1), use_idf=False, norm='l1')
vect.fit(df['pos_tag'])
vect = vect.transform(df['pos_tag'])
vect

<21072x32345 sparse matrix of type '<class 'numpy.float64'>'
	with 1459384 stored elements in Compressed Sparse Row format>

In [17]:
from sklearn.model_selection import train_test_split
y = df['categoria']
x_train, x_test, y_train, y_test = train_test_split(vect, y, test_size=0.25, random_state=42)

model = Sequential()

model.add(Dense(64, activation='relu', input_dim=32345))
model.add(Dropout(.3))
model.add(Dense(32, activation='relu'))
model.add(Dropout(.3))
model.add(Dense(5, activation='softmax'))

model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=3e-5), metrics="accuracy")

es = EarlyStopping(monitor='val_loss', verbose=1, patience=30)
mc = ModelCheckpoint('best_model_final_1.h5', monitor='val_loss', mode='min', save_best_only=True)
model.fit(np.asarray(x_train.todense()), pd.get_dummies(y_train.values).values,
          validation_data=(np.asarray(x_test.todense()), pd.get_dummies(y_test.values).values), 
          epochs=500,
          callbacks=[es, mc])

Epoch 1/500
494/494 [==============================] - 3s 7ms/step - loss: 1.6023 - accuracy: 0.2815 - val_loss: 1.5927 - val_accuracy: 0.2557
Epoch 2/500
494/494 [==============================] - 2s 5ms/step - loss: 1.5830 - accuracy: 0.3136 - val_loss: 1.5700 - val_accuracy: 0.2802
Epoch 3/500
494/494 [==============================] - 2s 5ms/step - loss: 1.5594 - accuracy: 0.3514 - val_loss: 1.5429 - val_accuracy: 0.3288
Epoch 4/500
494/494 [==============================] - 2s 5ms/step - loss: 1.5308 - accuracy: 0.4117 - val_loss: 1.5111 - val_accuracy: 0.4058
Epoch 5/500
494/494 [==============================] - 2s 5ms/step - loss: 1.4975 - accuracy: 0.4643 - val_loss: 1.4753 - val_accuracy: 0.5180
Epoch 6/500
494/494 [==============================] - 2s 5ms/step - loss: 1.4609 - accuracy: 0.5289 - val_loss: 1.4354 - val_accuracy: 0.6031
Epoch 7/500
494/494 [==============================] - 2s 5ms/step - loss: 1.4189 - accuracy: 0.5773 - val_loss: 1.3909 - val_accuracy: 0.6587

Epoch 58/500
494/494 [==============================] - 2s 5ms/step - loss: 0.3350 - accuracy: 0.8966 - val_loss: 0.3514 - val_accuracy: 0.8850
Epoch 59/500
494/494 [==============================] - 2s 5ms/step - loss: 0.3305 - accuracy: 0.8996 - val_loss: 0.3483 - val_accuracy: 0.8861
Epoch 60/500
494/494 [==============================] - 2s 4ms/step - loss: 0.3225 - accuracy: 0.9027 - val_loss: 0.3455 - val_accuracy: 0.8861
Epoch 61/500
494/494 [==============================] - 2s 4ms/step - loss: 0.3205 - accuracy: 0.9015 - val_loss: 0.3423 - val_accuracy: 0.8859
Epoch 62/500
494/494 [==============================] - 2s 5ms/step - loss: 0.3106 - accuracy: 0.9032 - val_loss: 0.3396 - val_accuracy: 0.8878
Epoch 63/500
494/494 [==============================] - 2s 5ms/step - loss: 0.3046 - accuracy: 0.9073 - val_loss: 0.3368 - val_accuracy: 0.8878
Epoch 64/500
494/494 [==============================] - 2s 5ms/step - loss: 0.3034 - accuracy: 0.9059 - val_loss: 0.3339 - val_accuracy:

Epoch 115/500
494/494 [==============================] - 2s 4ms/step - loss: 0.1486 - accuracy: 0.9589 - val_loss: 0.2981 - val_accuracy: 0.9009
Epoch 116/500
494/494 [==============================] - 2s 5ms/step - loss: 0.1457 - accuracy: 0.9611 - val_loss: 0.2987 - val_accuracy: 0.9002
Epoch 117/500
494/494 [==============================] - 2s 5ms/step - loss: 0.1459 - accuracy: 0.9603 - val_loss: 0.2990 - val_accuracy: 0.8983
Epoch 118/500
494/494 [==============================] - 2s 5ms/step - loss: 0.1421 - accuracy: 0.9609 - val_loss: 0.2994 - val_accuracy: 0.8992
Epoch 119/500
494/494 [==============================] - 2s 4ms/step - loss: 0.1395 - accuracy: 0.9631 - val_loss: 0.2996 - val_accuracy: 0.8988
Epoch 120/500
494/494 [==============================] - 2s 5ms/step - loss: 0.1402 - accuracy: 0.9609 - val_loss: 0.3000 - val_accuracy: 0.8992
Epoch 121/500
494/494 [==============================] - 2s 5ms/step - loss: 0.1405 - accuracy: 0.9611 - val_loss: 0.3003 - val_ac

In [18]:
rotulos = {"Cartão de crédito / Cartão pré-pago": 0, "Hipotecas / Empréstimos": 1, "Outros": 2, 
           "Roubo / Relatório de disputa": 3, "Serviços de conta bancária": 4}
y_test = y_test.map(rotulos)
y_train = y_train.map(rotulos)

In [21]:
from tensorflow.keras.models import load_model
model = load_model('best_model_final_1.h5')

train_preds = [np.argmax(result) for result in model.predict(x_train)]
test_preds = [np.argmax(result) for result in model.predict(x_test)]

gera_resultado(train_preds, y_train, test_preds, y_test)

Treino: 0.978426203469646
Teste: 0.8993974679151783


In [22]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, test_preds)

0.8993925588458618

In [23]:
y = df['categoria']
x_train, x_test, y_train, y_test = train_test_split(vect, y, test_size=0.25, random_state=42)

In [24]:
x_train = np.asarray(x_train.todense()).reshape(x_train.shape[0], x_train.shape[1], 1)
x_test = np.asarray(x_test.todense()).reshape(x_test.shape[0], x_test.shape[1], 1)

In [35]:
from tensorflow.keras.layers import SimpleRNN

In [40]:
model = Sequential()

# model.add(Embedding(32655, 128,))
model.add(LSTM(32, input_shape=(32345, 1), return_sequences=False))
# model.add(Dropout(.3))
# model.add(LSTM(32))
# model.add(Dropout(.3))
model.add(Dense(5, activation='softmax'))


In [41]:
model.compile(loss="categorical_crossentropy", optimizer='adam', metrics="accuracy")

es = EarlyStopping(monitor='val_loss', verbose=1, patience=18)
mc = ModelCheckpoint('best_model_final_2.h5', monitor='val_loss', mode='min', save_best_only=True)
model.fit(x_train, pd.get_dummies(y_train.values).values,
          validation_data=(x_test, pd.get_dummies(y_test.values).values),
          epochs=30,
          callbacks=[es, mc])

Epoch 1/30
494/494 [==============================] - 420s 850ms/step - loss: 1.5766 - accuracy: 0.2407 - val_loss: 1.5689 - val_accuracy: 0.2473
Epoch 2/30
189/494 [==========>...................] - ETA: 3:41 - loss: 1.5769 - accuracy: 0.2397

KeyboardInterrupt: 

In [35]:
from tensorflow.keras.layers import SimpleRNN

In [44]:
model = Sequential()

# model.add(Embedding(32655, 128,))
model.add(SimpleRNN(64, input_shape=(31821, 1), return_sequences=True, activation='relu'))
model.add(Dropout(.3))
model.add(SimpleRNN(32, activation='relu'))
model.add(Dropout(.3))
model.add(Dense(5, activation='softmax'))

In [45]:
model.compile(loss="categorical_crossentropy", optimizer='adam', metrics="accuracy")

es = EarlyStopping(monitor='val_loss', verbose=1, patience=18)
mc = ModelCheckpoint('best_model_final_2.h5', monitor='val_loss', mode='min', save_best_only=True)
model.fit(x_train, pd.get_dummies(y_train.values).values,
          validation_data=(x_test, pd.get_dummies(y_test.values).values), 
          epochs=30,
          callbacks=[es, mc])

Epoch 1/30
  4/494 [..............................] - ETA: 5:02:50 - loss: 1.6080 - accuracy: 0.2422

KeyboardInterrupt: 